### Sentiment Model Deployment
We will begin the process by deploying the sentiment model server. <br>
First download the model file using the following command:

In [1]:
! wget https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

--2020-08-20 08:27:02--  https://iguazio-sample-data.s3.amazonaws.com/models/model.pt
Resolving iguazio-sample-data.s3.amazonaws.com (iguazio-sample-data.s3.amazonaws.com)... 52.216.18.152
Connecting to iguazio-sample-data.s3.amazonaws.com (iguazio-sample-data.s3.amazonaws.com)|52.216.18.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433298364 (413M) [application/x-www-form-urlencoded]
Saving to: ‘model.pt’

model.pt            100%[===================>] 413.22M  12.2MB/s    in 35s     

2020-08-20 08:27:37 (11.9 MB/s) - ‘model.pt’ saved [433298364/433298364]



In [2]:
! mkdir /User/models/
! mv model.pt /User/models/model.pt

Next, we will import the sentiment server from mlrun's function hub.

In [7]:
from mlrun import import_function, mount_v3io
server_fn = import_function(url='hub://sentiment_analysis_serving')
server_fn.apply(mount_v3io())
server_fn.metadata.labels = {}

Finally, deploy it to the cluster.

In [8]:
server_fn.deploy(project='stocks')

> 2020-08-30 04:17:28,194 [info] deploy started
[nuclio] 2020-08-30 04:17:29,263 (info) Build complete
[nuclio] 2020-08-30 04:17:53,811 (info) Function deploy complete
[nuclio] 2020-08-30 04:17:53,816 done updating stocks-sentiment-analysis-server, function address: 18.217.172.163:30734


'http://18.217.172.163:30734'

Please note the http endpoint above. We will use it later on.